In [1]:
!git clone https://github.com/mymusise/ChatGLM-Tuning.git
%cd  ChatGLM-Tuning
!pip install -r requirements.txt 

Cloning into 'ChatGLM-Tuning'...
remote: Enumerating objects: 163, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 163 (delta 74), reused 65 (delta 61), pack-reused 51
Receiving objects: 100% (163/163), 8.03 MiB | 9.80 MiB/s, done.
Resolving deltas: 100% (86/86), done.
/content/ChatGLM-Tuning
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-g_oz52ez
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-g_oz52ez
  Resolved https://github.com/huggingface/peft.git to commit 632997d1fb776c3cf05d8c2537ac9a98a7ce9435
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
lora_path = "/content/drive/MyDrive/Colab Notebooks/output/"

In [4]:
from transformers import AutoModel
import torch


model = AutoModel.from_pretrained(
        "THUDM/chatglm-6b", load_in_8bit=True, trust_remote_code=True, device_map="auto"
    )

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/lib64-nvidia did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce562b961b3c22d803539adc1e0c19/gpu-t4-hm-2dc

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [5]:
from peft import PeftModel

model = PeftModel.from_pretrained(model, lora_path)

In [14]:
# 两种load方式，要和训练端一致
# from peft import get_peft_model, LoraConfig, TaskType

# peft_config = LoraConfig(
#     task_type=TaskType.CAUSAL_LM, inference_mode=True,
#     r=8,
#     lora_alpha=32, lora_dropout=0.1
# )

# model = get_peft_model(model, peft_config)
# model.load_state_dict(torch.load(lora_path + "chatglm-lora.pt"), strict=False)
# torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [12]:
from cover_alpaca2jsonl import format_example
def evaluate(instruction, input=None):
  with torch.no_grad():
    feature = format_example({'instruction': f'{instruction}', 'output': '', 'input': ''})
    input_text = feature['context']
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    out = model.generate(
            input_ids=input_ids,
            max_length=150,
            temperature=0
        )
    answer = tokenizer.decode(out[0])
    print(answer)

In [10]:
evaluate(input("instruction:"))

instruction:今天中午吃什么好


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1374: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Instruction: 今天中午吃什么好
Answer: 这完全取决于个人口味和饮食偏好。如果喜欢中餐,可以考虑吃炒菜、煲汤或煮粥等。如果喜欢西餐,可以考虑吃汉堡、披萨或意面等。也可以考虑咨询当地的餐厅,看看他们有什么推荐菜肴。无论选择什么,确保饮食均衡,摄入足够的营养,保持身体健康。
